### Gradient Boosting

### O que é o Gradinent Boosting?

O gradient boosting é uma técnica de aprendizado de máquina de múltiplas iterações, cujo objetivo é realizar a previsão de uma variável, que pode ser uma categoria, e portanto um algoritmo de classificação, ou uma variável contínua, e, portanto, um algoritmo de regressão.

### Embora ambos sejam algoritmos de Boosting, podemos listar muitas diferenças entre o Ada Boosting e o Gradient Boosting, vejamos algumas:
* Ambos são baseados em árvores de regressão. No entanto, o Ada Boosting utiliza apenas stumps, enquanto o Gradient Boosting utiliza árvores completas
* No Ada Boosting, cada árvore possui pesos diferentes para a decisão final baseado em seu índice de acerto. No Gradient Boosting, todas as árvores possuem igual impacto sobre a decisão final.
* A atribuição da primeira previsão de um algoritmo Ada Boost é feita usando o primeiro stump, já a do Gradient Boosting é feita utilizando a média dos valores a serem previstos.
* O Ada Boosting dificilmente overfitta em qualquer cenário, embora hajam exceções. Já o Gradient Boosting, caso não haja prudência ao selecionar o parâmetro Eta (learning_rate) é capaz de overfittar terrívelmente em 2 ou 3 iterações (para eta = 1, por exemplo).
* Enquanto os algoritmos anteriores se preocupavam em prever especificamente a variável dependente, o Gradient Boosting se preocupa em prever o resíduo da etapa anterior, sendo a previsão da primeira etapa feita conforme descrito acima.

### Implementação:

In [5]:
### Importanto bibliotecas:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2


In [7]:
### Carregando dados:
X, y = make_hastie_10_2(random_state=40028922)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

# Implementando o algoritmo:
clf = HistGradientBoostingClassifier(max_iter = 100).fit(X_train, y_train)
clf.score(X_test, y_test)

0.9008

### Implementação na dataset load_iris:

In [23]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


In [17]:
X, y = load_iris(return_X_y=True)
irisX_train, irisX_test, irisy_train, irisy_test = train_test_split(X, y, test_size=.25)

In [29]:
clfiris = GradientBoostingClassifier(n_estimators=120, random_state=40028922, validation_fraction=0.1).fit(irisX_train, irisy_train)
clfiris.score(irisX_test, irisy_test)

0.9210526315789473

### Sobre nossa implementação acima, algumas coisas são notáveis:
Utilizamos uma baixíssima quantidade de hiperparâmetros na nossa implementação. Seriam nossos hiper-parâmetros os melhores possíveis? Seria testar cada combinação manualmente uma opção viável na busca por um melhor modelo? A resposta é não. No entanto, existe uma ferramenta que permite que essa procura seja automática, que é o GridSearchCV. Assim, então, implementaremos novamente desta vez utilizando o GridSearch...

In [40]:
### Importando biblioteca:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

n_estim = list(range(50,250, 20))

In [38]:
model = GradientBoostingClassifier()
param_grid = {
    'n_estimators': n_estim,
    'max_depth': [None, 10, 20, 25],
    'min_samples_split': [2,4,6,8,10,12]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(irisX_train, irisy_train)

print('Os melhores parâmetros encontrados foram: ', grid_search.best_params_)


Os melhores parâmetros encontrados foram:  {'max_depth': None, 'min_samples_split': 6, 'n_estimators': 70}


### Assim sendo, avaliamos:

In [47]:
irisy_pred = grid_search.predict(irisX_test)
print('A acurácia do nosso modelo otimizado pelo GridSearch foi: ', accuracy_score(irisy_test, irisy_pred))


A acurácia do nosso modelo otimizado pelo GridSearch foi:  0.9210526315789473


### A diferença inexistente entre os scores se deve, provavelmente, a um misto de a base de dados ser muito pequena, e não haver diferença notável entre os parâmetros selecionados no nosso próprio modelo e o selecionado pelo GridSearch...

### Hiper-parametros importantes do Gradient Boosting:
* n_estimators: indica quantas tentativas recorrentes teremos de prever o erro que obtivemos na previsão anterior, ou seja, basicamente, indica o número de iterações do algoritmo.
* validation_fraction: indica qual será o percentual da base utilizada para validação
* n_iters_no_change: é um parâmetro usado para decidir se e sob qual condição as iterações devem ser interrompidas caso não haja alterações no score. Basicametne, estabelece um critério de parada precoce em caso de estagnação do score do algoritmo.
* ccp_alpha: como de costume, é um parâmetro utilizado para estipular o grau de complexidade permitido em nossas árvores de regressão
* learning_rate: é o nosso "eta" do GBoosting. Basicamente, esse parâmetro restringe a contribuição de cada iteração em direção do valor real, de modo a evitar uma convergência precoce e consequente overfitting.
* verbose: cria uma descrição da execução do algoritmo, demonstrando as etapas que estão sendo executadas. É importante, principalmente para quando, por exemplo, o algoritmo estiver rodando em uma base de dados muito grande e sua execução for demasiado demorada, levantando dúvidas quanto à se o algoritmo ainda está em execução ou se aconteceu algum "crash"

### Até agora, utilizamos o Gradient Boosting apenas em problemas de Classificação, mas ele também se aplica em regressão, como abaixo:

In [51]:
### Importando bibliotecas:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_friedman1
from sklearn.metrics import mean_squared_error
import numpy as np


In [53]:
X_friedman, y_friedman = make_friedman1(n_samples=1200, random_state=40028922, noise=1.0)
trainX_friedman, testX_friedman = X_friedman[:200], X_friedman[200:]
trainy_friedman, testy_friedman = y_friedman[:200], y_friedman[200:]
est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=40028922, loss='squared_error'
).fit(trainX_friedman, trainy_friedman)
mean_squared_error(testy_friedman, est.predict(testX_friedman))


5.748054670292009